### Test bench for the communication protocol as of Oct 2, 2025. 

In [4]:
pip install crcmod


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for crcmod: filename=crcmod-1.7-py3-none-any.whl size=18869 sha256=659ea352866ec1ae5ee6cd822bf9a9716f771b9e0ce17371c02c647a63407525
  Stored in directory: c:\users\roths\appdata\local\pip\cache\wheels\23\94\7a\8cb7d14597e6395ce969933f01aed9ea8fa5f5b4d4c8a61e99
Successfully built crcmod
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'crcmod' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'crcmod'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\roths\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
from protocol import pack, unpack, START_OF_FRAME
import struct

import crcmod

# Helper function to print bytes in a readable hex format
def pretty_hex(data: bytes) -> str:
    return ' '.join(f'{b:02X}' for b in data)

In [ ]:
left_speed = 0.75
right_speed = -0.5
test_payload = struct.pack('>ff', left_speed, right_speed)
#print(test_payload)

b'?@\x00\x00\xbf\x00\x00\x00'


In [8]:

print(f"Original Payload: {pretty_hex(test_payload)} (Floats: {left_speed}, {right_speed})")


Original Payload: 3F 40 00 00 BF 00 00 00 (Floats: 0.75, -0.5)


In [9]:
packet = pack(packet_type=0x01, sequence_number=101, payload=test_payload)
print(f"Full Packed Packet: {pretty_hex(packet)}")
print(f"Total Packet Length: {len(packet)} bytes\n")


Full Packed Packet: 1A CF 01 00 65 08 3F 40 00 00 BF 00 00 00 D3 B3
Total Packet Length: 16 bytes



## Unpacking


In [10]:
unpacked_data, consumed_bytes = unpack(packet)

print(f"Unpacked Data: {unpacked_data}")
print(f"Bytes Consumed: {consumed_bytes}\n")

Unpacked Data: {'type': 1, 'seq': 101, 'payload': b'?@\x00\x00\xbf\x00\x00\x00'}
Bytes Consumed: 16



In [12]:
assert unpacked_data is not None
assert unpacked_data['type'] == 0x01
assert unpacked_data['seq'] == 101
# Unpack the payload floats to verify them
unpacked_left, unpacked_right = struct.unpack('>ff', unpacked_data['payload'])
assert unpacked_left == left_speed
assert unpacked_right == right_speed
print("Test PASSED!\n")

Test PASSED!



In [13]:
corrupted_packet_list = list(packet)
# Corrupt a byte in the payload section (byte 8)
corrupted_packet_list[8] += 1 
corrupted_packet = bytes(corrupted_packet_list)

In [14]:
print(f"Original Packet:    {pretty_hex(packet)}")
print(f"Corrupted Packet:   {pretty_hex(corrupted_packet)}\n")


Original Packet:    1A CF 01 00 65 08 3F 40 00 00 BF 00 00 00 D3 B3
Corrupted Packet:   1A CF 01 00 65 08 3F 40 01 00 BF 00 00 00 D3 B3



In [15]:
unpacked_data, consumed_bytes = unpack(corrupted_packet)
print(f"Unpacked Data: {unpacked_data}")
print(f"Bytes Consumed: {consumed_bytes}\n")


Checksum mismatch! Received: 54195, Calculated: 55192
Unpacked Data: None
Bytes Consumed: 16



In [16]:
stream_buffer = b'\xDE\xAD\xBE\xEF' + packet + b'\xCA\xFE'
print(f"Full Stream Buffer: {pretty_hex(stream_buffer)}\n")

Full Stream Buffer: DE AD BE EF 1A CF 01 00 65 08 3F 40 00 00 BF 00 00 00 D3 B3 CA FE



In [18]:
unpacked_data, consumed_bytes = unpack(stream_buffer)
print(f"Unpacked Data: {unpacked_data}")
print(f"Total Bytes Consumed from buffer start: {consumed_bytes}\n")


Unpacked Data: {'type': 1, 'seq': 101, 'payload': b'?@\x00\x00\xbf\x00\x00\x00'}
Total Bytes Consumed from buffer start: 20



In [20]:
assert unpacked_data is not None
assert unpacked_data['type'] == 0x01
# We consumed 4 garbage bytes + the packet length
assert consumed_bytes == 4 + len(packet)
print("Garbage Data Stream Test PASSED! (Packet was successfully found and parsed)\n")


Garbage Data Stream Test PASSED! (Packet was successfully found and parsed)

